# Practical 2: Forecasting Housing Prices
<hr>

In [ ]:
# Standard imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## The California Housing data set

The "California Housing" data set is a widely used data set to demonstrate forecasting methods. As the name suggests, the data set concerns the valuation of real estate. It comprises socio-demographic information concerning the area of a property and a dependent (aka target) variable, which gives the median house value for California districts. This data was derived from the 1990 U.S. census. 

Being so popular, the data set is readily available in standard Python libraries. Execute the following code to load the data set from `sklearn.datasets` and familiarize yourself with the data description.

In [ ]:
# Downloading the data set
from sklearn.datasets import fetch_california_housing

california_housing = fetch_california_housing(as_frame=True)  # get the data as a Pandas dataframe

print(california_housing.DESCR)  # the data even comes with a nice description

The `sklearn` library provides the data in a specific format. Feature values and the target variable are already separated. The be consistent with our standard notation, we extract the feature values and target and store it, as usual, in variables $X$ and $y$, respectively. Of course, this is a good opportunity to also take a quick look into the data.

In [ ]:
X = california_housing.data
X.head(5)

In [ ]:
y = california_housing.target
y

### Descriptive Statistics

In [ ]:
y.describe()  # compute descriptive statistics that summarize the target's distribution

In [ ]:
X.describe() # do the same for all features

### Explanatory data analysis (EDA)
To better understand the data, we consider a few standard EDA operations. They comprise the analysis of histograms and/or box-plots of the target and the features. Also, we consider the (linear) correlation between features and the target to obtain some initial evidence as to which features might be important. 

While we only create the plots in the following, never forget that each plot - and more generally every result - deserves a careful analysis and discussion. Therefore, make sure to examine each plot and note down your key observations.

Without any strong reason, we use the `seaborn` library for plotting in this notebook. Many data scientists prefer `seaborn` over `Matplotlib`. Hence, we illustrate it here.

#### Target Variable: Medium House Value

##### Analyzing the distribution of the target by means of a histogram

In [ ]:
# Histogram of target
plt.figure(figsize=(8, 6))
sns.histplot(y, bins=30, kde=False, edgecolor='k')
plt.xlabel(y.name)
plt.ylabel('Frequency')
plt.title(f'Distribution of {y.name}')
plt.show()


##### Analyzing the distribution of the target by means of a boxplot

In [ ]:
sns.boxplot(x=y, color='skyblue');

#### Features

In [ ]:
#3x3 matrix of box-plots for all the features
fig, axes = plt.subplots(3, 3, figsize=(15, 12))  # Create a 3x3 grid of subplots
axes = axes.flatten() # Flatten the axes for easier iteration

# Loop through each feature and create a box plot
for i, feature in enumerate(X.columns):
    sns.boxplot(x=X[feature], ax=axes[i], color='skyblue')
    axes[i].set_title(f'Box Plot of {feature}')
    axes[i].set_xlabel(feature)

# Remove empty subplots
for i in range(len(X.columns), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()  # Adjust layout
plt.show()

#### Correlation analysis
Having obtained an idea about the *univariate* distributions, we also take a look at the correlation structure in the data.

In [ ]:
# Correlation among the features
corr_matrix = X.corr()  
sns.heatmap(corr_matrix, cmap='vlag', annot=True)
plt.title('Feature correlation')
plt.show()

In [ ]:
# Correlation between individual features and the target
rho = X.corrwith(y)
print(rho)

sns.barplot(x=rho, y=X.columns.tolist(), orient='h')
plt.title('Feature to target correlation')
plt.xlabel('Correlation')
plt.show()

## Linear Regression
Having completed our initial data screening and explorative analysis, we proceed by estimating a linear regression model to deepen our understanding of how the features and the target are related to another.

To that end, we consider the library `statsmodels`, a popular and powerful library for statistical modeling, which includes ordinary least squares (OLS) estimator. We provide all codes for the model estimation below.

In [ ]:
import statsmodels.api as sm 

# OLS model estimation
lr_sm = sm.OLS(y,
               sm.add_constant(X)  # include an intercept
              ) 

lr_sm = lr_sm.fit()  # fit the model

The library provides a neat function, `.summary()` to obtain a concise overview of the results of regression analysis. It includes key information like R-squared, estimated coefficients, standard errors, and p-values. This summary is crucial for evaluating model adequacy and feature significance.

In [ ]:
print(lr_sm.summary())

# Exercises
Based on the above analysis, draw on your data science expertise to answer the following questions: 

## 1. What are the two most and the two least important features?
Briefly note what statistics/results you have considered to make your choice.

**Enter your answer in this markup cell:**

Suitable statistics to clarify the importance of individual features are the T-Test and the feature-to-target correlation.

Considering the **T-test**, the two most important features are:
- MedInc
- Latitude

and the two least important features are:
- Population
- AveOccup

Considering the **Feature-to-target correlation**, the two most important features are:
- MedInc
- AveRooms

and the two least important features are:
- Population
- AveOccup




## 2. Scatter Plots
Create, for each of the selected features, a scatter plot of the selected feature against the target. Display these plots in a 2x2 grid

In [ ]:
# Code for plotting 
features_to_plot = ['MedInc', 'AveRooms', 'Population', 'AveOccup']  # Selection based on correlation

# Create a 2x2 grid of scatter plots
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
fig.suptitle("Scatter Plots of Most/Least Important Features vs. Y", fontsize=16)

# Iterate through features and plot using Seaborn
for i, column in enumerate(features_to_plot):
    row, col = divmod(i, 2)
    sns.scatterplot(x=X[column], y=y, ax=axes[row, col], alpha=0.7)
    axes[row, col].set_title(f"{column} vs Y")
    axes[row, col].set_xlabel(column)
    axes[row, col].set_ylabel("Y")

plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to fit title
plt.show()


## 3. Model Reestimation
Remove the two least important features from the data and reestimate the model. Briefly discuss whether this step has improved the model.

In [ ]:
# Code to reestimate the model:

# Remove least important features (based on correlation)
X_sparse = X.drop(columns=['AveOccup', 'Population'])

# Reestimate the model
sparse_model = sm.OLS(y,
               sm.add_constant(X_sparse)  # include an intercept
              ) 

sparse_results = sparse_model.fit()  # fit the modelX_sparse

print(sparse_results.summary())

**Did the model improve? Briefly discuss:**

Overall, differences between the models are minor. We could, for example, compare the $R^2$ statistics as shown below. Of course, you can also consider other statistics such as, e.g.,  *AIC*.

In [ ]:
print(f'R2 when using all features: {lr_sm.rsquared_adj:.5f}')
print(f'R2 when discarding two least important features: {sparse_results.rsquared_adj:.5f}')

## 4. Switching the Library
We have used `statsmodels` up to this point. However, the go-to library when it comes to machine learning is a different library called [scikit-learn](https://scikit-learn.org/stable/), typically abbreviated as `sklearn`.

Import that library. Then, using once more the data set with all features included, create another linear regression model using the class `linear_model.LinearRegression()`. Compare the coefficients between this model and the one we estimated above using `statsmodels.api.OLS`. They should be pretty much identical. Please verify that they are.

In [ ]:
# Code to estimate linear regression using sklearn.
from sklearn.linear_model import LinearRegression

lr_sk = LinearRegression(fit_intercept=True)
lr_sk.fit(X=X, y=y)


In [ ]:
# Code to compare regression coefficients
# -----------------------------------------

# Extract coefficients from statsmodels
sm_params = lr_sm.params.values
sm_params_names = lr_sm.params.index

# Extract coefficients from sklearn
# Sklearn stores intercept separately, so we concatenate it with coef_
sklearn_params = [lr_sk.intercept_] + list(lr_sk.coef_)
sklearn_params_names = ['Intercept'] + list(lr_sm.params.index[1:])

# Combine into a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Statsmodels Coefficients': sm_params,
    'Sklearn Coefficients': sklearn_params
}, index=sm_params_names)

# Display the DataFrame to compare coefficients side-by-side
print(comparison_df)

## 5. Prediction 
We next use our regression model for prediction. Feeding it with data on feature values, the estimated regression coefficients facilitate forecasting real-estate prices. 

For start, compute forecasts for the training data, that is, the data from which you estimated the regression coefficients. Store your forecasts in a variable, `y_hat`, and assess them in terms of mean squared error (MSE), defined as: <br><br>
$ MSE = \frac{1}{n}\sum_{i=1}^{n} \left( Y_i - \hat{Y}_i \right)^2 $,
<br><br>
with:
- $n$ = number of data points
- $Y$ = true values of the target variable
- $\hat{Y}$ = forecasts of the regression model

> Hint: if unsure how to implement the MSE yourself, you can be sure that ready-made functions are available to do it for you ;)


In [ ]:
# Code to assess regression model forecasts using MSE
y_hat = lr_sk.predict(X)
mse = np.sum((y-y_hat)**2)/len(y)
mse

In [ ]:
# Alternative solution using sklearn function
from sklearn.metrics import mean_squared_error
mean_squared_error(y_true=y, y_pred=y_hat)

## Closing questions
- We assessed the model in terms of MSE. Why is the root of the MSE often preferred in practice?
- Beyond (R)MSE, considering all results observed in the notebook, what is your overall opinion of the regression model?
- No model is perfect. What could we try to improve the value of the regression model for forecasting?

**Space to take notes on the above points**


RMSE is preferred because it has the same scale as the target. For example, if the target is measured in EUR, than RMSE gives the EUR-amount by which the forecast is, on average, away from the target. This quantity is easier to interpret than the squared average difference.

Overall, the regression model is doing a fair job. This can be deduced from the $R^2$ statistic of roughly 60%. From the EDA, we can tell that the distribution of the target is non-normal, which is a red flag for linear regression. The reason is that, for reasons of confidentiality, the target variable is clipped at a value of 5. 

Another issue concerns the features, which exhibit several outliers. For example, the scatter plots of predictive features versus the target (Exercise 2) indicate the adverse effect of outliers. 

Given we cannot do anything about the odd measurement of the target, removing or truncating outliers among the features is a good first step to improve the regression model. 

# EXTRA Exercise (Challenging!):

## Backward Elimination 
We examined the effect of discarding features in a previous exercise. This task takes a more comprehensive approach toward feature selection.

Relying on the following pseudo code, implement a *backward elimination* procedure, in which you repetitively discard the least important feature from the regression model.

```
    Use all features as the current set of features  
    Do 
        Estimate a regression model using current set of features
        Store model performance in terms of a suitable statistic
        Identify the least important feature
        Discard that feature from current set of features
    Repeat until all features but one have been deleted
   
```
Depict your results graphically by plotting the number of features in the regression model against model performance using the same statistic as in your backward elimination algorithm.
>Hint: we recommend you use the function `OLS` from the  `statsmodels` library for this exercise. 

In [ ]:
# Code for your backward elimination algorithm:
#-------------------------------------------------------
# Initialize the current set of features
current_features = list(X.columns)
model_performance = []

# Begin backward elimination
while len(current_features) > 1:
    # Add a constant to the model (intercept) and fit with statsmodels for p-values
    X_current = sm.add_constant(X[current_features])
    model = sm.OLS(y, X_current).fit()
    
    # Store model performance in terms of R-squared and current features
    r2 = model.rsquared
    model_performance.append((current_features.copy(), r2, model.summary()))

    # Find the feature with the highest p-value (least statistically significant)
    p_values = model.pvalues.iloc[1:]  # Exclude the intercept's p-value
    least_significant_feature = p_values.idxmax()  # Get feature with highest p-value

    # Remove the least significant feature from the current set
    current_features.remove(least_significant_feature)

# Convert performance results to DataFrame for easier inspection
performance_df = pd.DataFrame([(features, r2) for features, r2, _ in model_performance], 
                              columns=['Features', 'R-squared'])
print(performance_df)

In [ ]:
# Code for visualizing the results of backward elimination
# ----------------------------------------------------------------

# Extract the number of features and corresponding R-squared values for the plot
num_features = [len(features) for features, r2, _ in model_performance]
r2_values = [r2 for features, r2, _ in model_performance]

# Plot R-squared values against the number of features
plt.figure(figsize=(8, 6))
plt.plot(num_features, r2_values, marker='o')
plt.xlabel("Number of Features")
plt.ylabel("R-squared")
plt.title("R-squared vs. Number of Features in Backward Elimination")
plt.gca().invert_xaxis()  # Invert x-axis to show decreasing number of features
plt.grid()
plt.show()